In [1]:
from collections import Counter
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
#import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
directory = './brunch-article-recommendation/res/'

# dev 및 test user 불러오기
dev_users, test_users = [], []

with open(directory + 'predict/dev.users', 'r') as fr:
    lines = fr.readlines()
    dev_users = lines
    del lines
for i in range(len(dev_users)):
    dev_users[i] = dev_users[i].replace('\n','')
    
with open(directory + 'predict/test.users', 'r') as fr:
    lines = fr.readlines()
    test_users = lines
    del lines
for i in range(len(test_users)):
    test_users[i] = test_users[i].replace('\n','')

print(len(dev_users))
print(dev_users[:5])
print(len(test_users))
print(test_users[:5])

3000
['#d6866a498157771069fdf15361cb012b', '#f963fb8c5d9d14d503fc4e80bd8617b4', '#87a6479c91e4276374378f1d28eb307c', '#677e984e245b344f61dc5d3cc1f352c8', '#519f45eb14e4807e8714fb7e835463eb']
5000
['#7ee14df8642a7925b1465ff5c89efe5b', '#8420b9385b282028eebf1ad6b4a221c0', '#c9b31d8b64357f5854b1ba55b32eb6d3', '#9bb1e13b5481fa3737af20870b25c723', '#37d5f99a7f12c9ba90c4e2ac92e54ab6']


In [2]:
import time
from tqdm import tqdm

In [3]:
start_time = time.time()
read_file_lst = glob.glob('./brunch-article-recommendation/res/read/*')
exclude_file_lst = ['read.tar']
read_df_lst = []
for f in read_file_lst:
    file_name = os.path.basename(f)
    if file_name in exclude_file_lst:
        print(file_name)
    else:
        df_temp = pd.read_csv(f, header=None, names=['raw'])
        df_temp['dt'] = file_name[:8]
        df_temp['hr'] = file_name[8:10]
        df_temp['user_id'] = df_temp['raw'].str.split(' ').str[0]
        df_temp['article_id'] = df_temp['raw'].str.split(' ').str[1:].str.join(' ').str.strip()
        read_df_lst.append(df_temp)
        
read = pd.concat(read_df_lst)
print ("%s sec"%(time.time() - start_time))

41.336071491241455 sec


In [4]:
def chainer(s):
    return list(chain.from_iterable(s.str.split(' ')))
read_cnt_by_user = read['article_id'].str.split(' ').map(len)
read_raw = pd.DataFrame({'dt': np.repeat(read['dt'], read_cnt_by_user),
                         'hr': np.repeat(read['hr'], read_cnt_by_user),
                         'user_id': np.repeat(read['user_id'], read_cnt_by_user),
                         'article_id': chainer(read['article_id'])})
read_raw.shape



(22110706, 4)

# 아래 맞게수정필요!!

In [5]:
metadata = pd.read_json(directory + 'metadata.json', lines=True) 

In [6]:
atc = metadata.copy()
atc['reg_datetime'] = atc['reg_ts'].apply(lambda x : datetime.fromtimestamp(x/1000.0))
atc.loc[atc['reg_datetime'] == atc['reg_datetime'].min(), 'reg_datetime'] = datetime(2090, 12, 31)
atc['reg_dt'] = atc['reg_datetime'].dt.date
atc['type'] = atc['magazine_id'].apply(lambda x : '개인' if x == 0.0 else '매거진')
# 컬럼명 변경
atc.columns = ['id', 'display_url', 'article_id', 'keyword_list', 'magazine_id', 'reg_ts', 'sub_title', 'title', 'author_id', 'reg_datetime', 'reg_dt', 'type']
atc.head()
atc_cnt_by_reg_dt = atc.groupby('reg_dt', as_index=False)['article_id'].count()

In [7]:
atc_read_cnt = read_raw[read_raw.article_id != ''].groupby('article_id')['user_id'].count()
atc_read_cnt = atc_read_cnt.reset_index()
atc_read_cnt.columns = ['article_id', 'read_cnt']
#metadata 결합
atc_read_cnt = pd.merge(atc_read_cnt, atc, how='left', left_on='article_id', right_on='article_id')
# metadata를 찾을 수 없는 소비 로그 제외
atc_read_cnt_nn = atc_read_cnt[atc_read_cnt['id'].notnull()]
# 소비수 기준 분류값
#atc_read_cnt_nn.sort_values(by='read_cnt', ascending=False).head(2)

In [8]:
# 소비수 기준 분류값
def get_class(x):
    if x >= 142:
        result = '5%'
    elif x >= 72:
        result = '10%'
    elif x >= 25:
        result = '25%'
    elif x >= 8:
        result = '50%'
    elif x >= 3:
        result = '75%'
    else:
        result = '100%'
    return result
atc_read_cnt_nn['class'] = atc_read_cnt_nn['read_cnt'].map(get_class)


D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [9]:
off_data = pd.merge(read_raw, atc, how='inner', left_on='article_id', right_on='article_id')

In [10]:
print(off_data.shape)
off_data.columns = ['read_dt', 'hr', 'user_id', 'article_id', 'article_seq', 'display_url',
                    'keyword_list', 'magazine_id', 'reg_ts', 'sub_title', 'title',
                    'author_id', 'reg_datetime', 'reg_dt', 'type']

(20905040, 15)


In [ ]:
off_data = off_data[['read_dt', 'user_id', 'article_id', 'title', 'sub_title', 'author_id', 'reg_dt', 'type', 'display_url', 'keyword_list', 'magazine_id']]
off_data['read_dt'] = pd.to_datetime(off_data['read_dt'], format='%Y%m%d')
off_data['reg_dt'] = pd.to_datetime(off_data['reg_dt'], format='%Y-%m-%d')
off_data['off_day'] = (off_data['read_dt'] - off_data['reg_dt']).dt.days
off_data = pd.merge(off_data, atc_read_cnt_nn[['article_id', 'read_cnt', 'class']], how='left', left_on='article_id', right_on='article_id')

In [ ]:
result = pd.DataFrame()
for i in tqdm(range(3000),mininterval=1):
    off_data_i = off_data[off_data['user_id'] == dev_users[i]].sort_values(by=['read_dt']).tail(100)
    result = pd.concat([result,off_data_i])

 11%|████████▏                                                                    | 319/3000 [15:44<2:13:39,  2.99s/it]

In [66]:
result.to_csv(directory+'rasby_devuser.csv',encoding='utf-8')

# 여기까지!

In [ ]:
# 9s
off_data_0 = off_data[off_data['user_id'] == dev_users_list[0]]
# 9s
off_data_1 = off_data[off_data['user_id'] == dev_users_list[1]]
off_data_1.sort_values(by=['read_dt']).tail(20)

In [68]:
with open(directory + 'rasby_devuser.csv',encoding='utf-8') as f:
    r1 = pd.read_csv(f)

In [ ]:
# 최근 한달간 읽은 글들만 모으기
a = 20190201
a = pd.to_datetime(a,format='%Y%m%d')
result = result[result['read_dt'] >= a]

# 2월 글 뽑기

In [ ]:
FEB_read_raw = read_raw[read_raw['dt'] >= '20190201']
len(FEB_read_raw)
FEB_atc_read_cnt = FEB_read_raw[FEB_read_raw.article_id != ''].groupby('article_id')['user_id'].count()
FEB_atc_read_cnt = FEB_atc_read_cnt.reset_index()
FEB_atc_read_cnt.columns = ['article_id', 'read_cnt']
FEB_atc_read_cnt = pd.merge(FEB_atc_read_cnt, atc, how='left', left_on='article_id', right_on='article_id')
# metadata를 찾을 수 없는 소비 로그 제외
FEB_atc_read_cnt_nn = FEB_atc_read_cnt[FEB_atc_read_cnt['id'].notnull()]
FEB_atc_read_cnt_nn['class'] = FEB_atc_read_cnt_nn['read_cnt'].map(get_class)
len(FEB_atc_read_cnt_nn)

In [80]:
import time
start_time = time.time()
atc_read_cnt = atc_read_cnt_nn[atc_read_cnt_nn['read_cnt'] > 20]
article_vocab_list = atc_read_cnt['article_id'].tolist()
print(len(article_vocab_list))
print ("%s sec"%(time.time() - start_time))

134991
0.22319436073303223 sec


In [106]:
TOTAL_top = atc_read_cnt.sort_values(by='read_cnt',ascending=False)
len(TOTAL_top)
TOTAL_top_30000 = TOTAL_top[:30000]

In [111]:
FEB_top = FEB_atc_read_cnt_nn.sort_values(by='read_cnt',ascending=False)
len(FEB_top)
FEB_top = FEB_top[FEB_top['read_cnt'] >= 15]
len(FEB_top)
FEB_top = FEB_top[:30000]

In [114]:
TOTAL_top_30000.to_csv(directory+'TOTAL_top_30000.csv',encoding = 'utf-8')
FEB_top.to_csv(directory + 'FEB_top_30000.csv',encoding='utf-8')

In [ ]:
# FEB_MAR 에 쓰여진 글

In [125]:
a = 20190201
a = pd.to_datetime(a,format='%Y%m%d')
FEB_MAR_article  = atc[atc['reg_datetime'] >= a]

In [126]:
len(FEB_MAR_article)

44278

In [129]:
FEB_MAR_article = FEB_MAR_article.sort_values(by='reg_datetime')
FEB_MAR_article.head(2)

In [131]:
FEB_MAR_article.to_csv(directory + 'FEB_MAR_article.csv',encoding='utf-8')